In [1]:
from transformers import pipeline , AutoModelForCausalLM, AutoTokenizer
import torch
import torch.nn.functional as F
from utils import tokenize_hfmodel_inputs
device='cuda' if torch.cuda.is_available() else 'cpu'

/home/Testing_diff_models/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
low_compute=['mistralai/Mistral-7B-v0.3','google/gemma-2b','microsoft/phi-2','meta-llama/Llama-2-7b-hf','teknium/OpenHermes-2.5-Mistral-7B','meta-llama/Llama-3.1-8B','microsoft/Phi-3-mini-128k-instruct']
instruct_counter_part=['mistralai/Mistral-7B-Instruct-v0.3']

Text completions

In [5]:
model_name=low_compute[2]

In [6]:
model=AutoModelForCausalLM.from_pretrained(model_name).to(device)

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.26s/it]


In [7]:
sample_text="How to mak a sandwich"


tokens,tokeniser=tokenize_hfmodel_inputs(sample_text,model_name,'cpu')
x=tokens.to(device)
for _ in range(50):
    ops=model(input_ids=x)
    logits=ops.logits
    probs=F.softmax(logits[:,-1,:],dim=-1)

    next_token=torch.multinomial(probs.view(probs.shape[1]),1)

    x=torch.cat([x,next_token.view(1,1)],dim=-1)
op=''
for i in x.tolist():
    op=op+tokeniser.decode(i)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/Testing_diff_models/utils.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokens=torch.tensor(tokens).view(1,tokens.shape[0])


In [8]:
print(op)

How to mak a sandwich). Toast both slices of bread. Spread mayonnaise and mustard on one slice of bread. Layer turkey, cheese, and lettuce on top. Place the other slice of bread on the top and voila! You have a delicious sandwich ready to be


Chat

In [10]:
from transformers import pipeline , AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList
import torch
import torch.nn.functional as F
from utils import tokenize_hfmodel_inputs
device='cuda' if torch.cuda.is_available() else 'cpu'

In [14]:
model_name="google/gemma-2b-it"

In [15]:
model=AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokeniser=AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.44it/s]


In [16]:
class StopOnEos(StoppingCriteria):
    def __call__(self, input_ids, scores, **kwargs):
        # Check if the last generated token is the EOS token
        eos_token_id = 50256  # Change this based on your model's tokenizer
        return input_ids[0, -1] == eos_token_id

stopping_criteria = StoppingCriteriaList([StopOnEos()])

In [19]:
sample_text="write a frontend code to an ecommerce application in react."
messages = [{"role": "user", "content": sample_text}]
prompt = tokeniser.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
input_ids = tokeniser(prompt, return_tensors="pt").to(device)
output_ids = model.generate(input_ids.input_ids,max_length=500,do_sample=True,temperature=0.7,top_p=0.9)
response = tokeniser.decode(output_ids[0], skip_special_tokens=True)
print(response)

user
write a frontend code to an ecommerce application in react.
model
```javascript
import React, { useState } from "react";
import "./App.css";

const App = () => {
  const [products, setProducts] = useState([]);

  const handleAddProduct = (product) => {
    setProducts([...products, product]);
  };

  return (
    <div className="app">
      <h1>Ecommerce App</h1>
      <input type="text" placeholder="Enter product name" />
      <input type="number" placeholder="Enter product price" />
      <button onClick={() => handleAddProduct({ name: "", price: 0 })}>
        Add Product
      </button>
      <ul>
        {products.map((product) => (
          <li key={product.id}>{product.name} - ${product.price}</li>
        ))}
      </ul>
    </div>
  );
};

export default App;
```

**CSS:**

```css
.app {
  display: flex;
  flex-wrap: wrap;
  margin: 0 auto;
  padding: 20px;
}
```

**Explanation:**

* The `useState` hook is used to manage the state of the products.
* The `handleAddProduc

In [ ]:
messages.append({"role": "assistant", "content": response})
messages.append({"role": "user", "content": "But you did not write c=a+b anywhere"})

prompt = tokeniser.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
input_ids = tokeniser(prompt, return_tensors="pt")

output_ids = model.generate(input_ids.input_ids, max_length=1200, do_sample=True, top_p=0.9)
response = tokeniser.decode(output_ids[0], skip_special_tokens=True)

print(response)


Code

In [ ]:
model_name="google/gemma-2b-it"

In [ ]:
sample_text="write a c program to print first 5 numbers of fibonacci series."
model=AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokeniser=AutoTokenizer.from_pretrained(model_name)
messages = [{"role": "user", "content": sample_text}]
prompt = tokeniser.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
input_ids = tokeniser(prompt, return_tensors="pt").to(device)
output_ids = model.generate(input_ids.input_ids,max_length=200,do_sample=True,temperature=0.7,top_p=0.9)
response = tokeniser.decode(output_ids[0], skip_special_tokens=True)
print(response)